# 准备数据

## 制作数据集

从 Hugging Face 下载数据集 `segments/sidewalk-semantic`，按 7:1:2 切分训练-验证-测试集，然后存放到 `./data/sidewalk-semantic` 目录下。

In [1]:
from datasets import load_dataset
from PIL import Image
from tqdm import tqdm
import os
import random

# 输出路径
OUTPUT_PATH = "./data/sidewalk-semantic"
image_path = os.path.join(OUTPUT_PATH, "images")
mask_path = os.path.join(OUTPUT_PATH, "masks")
os.makedirs(image_path, exist_ok=True)
os.makedirs(mask_path, exist_ok=True)

# 加载和保存数据集
dataset = load_dataset("segments/sidewalk-semantic", data_dir="data")["train"]
id = 0
for sample in tqdm(dataset):
    image = sample["pixel_values"]
    mask = sample["label"]
    image.save(os.path.join(image_path, f"{id}.jpg"))
    mask.save(os.path.join(mask_path, f"{id}.png"))
    id += 1

# 切分训练-验证-测试集，比例为7:1:2
ids = list(range(id))
random.shuffle(ids)
train_ids = ids[:int(id*0.7)]
val_ids = ids[int(id*0.7):int(id*0.8)]
test_ids = ids[int(id*0.8):]
with open(os.path.join(OUTPUT_PATH, "train.txt"), "w") as f:
    train_ids_str = [str(i) for i in train_ids]
    f.write("\n".join(train_ids_str))
with open(os.path.join(OUTPUT_PATH, "val.txt"), "w") as f:
    val_ids_str = [str(i) for i in val_ids]
    f.write("\n".join(val_ids_str))
with open(os.path.join(OUTPUT_PATH, "test.txt"), "w") as f:
    test_ids_str = [str(i) for i in test_ids]
    f.write("\n".join(test_ids_str))

# 保存标签
labels = [
    "unlabeled",
    "flat-road",
    "flat-sidewalk",
    "flat-crosswalk",
    "flat-cyclinglane",
    "flat-parkingdriveway",
    "flat-railtrack",
    "flat-curb",
    "human-person",
    "human-rider",
    "vehicle-car",
    "vehicle-truck",
    "vehicle-bus",
    "vehicle-tramtrain",
    "vehicle-motorcycle",
    "vehicle-bicycle",
    "vehicle-caravan",
    "vehicle-cartrailer",
    "construction-building",
    "construction-door",
    "construction-wall",
    "construction-fenceguardrail",
    "construction-bridge",
    "construction-tunnel",
    "construction-stairs",
    "object-pole",
    "object-trafficsign",
    "object-trafficlight",
    "nature-vegetation",
    "nature-terrain",
    "sky",
    "void-ground",
    "void-dynamic",
    "void-static",
    "void-uncle",
]
with open(os.path.join(OUTPUT_PATH, "labels.txt"), "w") as f:
    f.write("\n".join(labels))

c:\Users\18782\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (C:/Users/18782/.cache/huggingface/datasets/segments___parquet/segments--sidewalk-semantic-2-bb87025dee0a4a60/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1000/1000 [00:55<00:00, 18.17it/s]


## 计算均值和方差

In [2]:
from torch.utils.data import Subset
import numpy as np

# 获取训练集
train_set = Subset(dataset["pixel_values"], train_ids)

# 计算均值和方差
mean = 0
std = 0
for image in tqdm(train_set):
    image = np.array(image)
    mean += image.mean(axis=(0, 1))
    std += image.std(axis=(0, 1))
mean /= len(train_set)
std /= len(train_set)
print(mean, std)

100%|██████████| 700/700 [02:05<00:00,  5.58it/s]

[132.77355813 127.61624462 120.28149316] [58.87826301 56.10282104 59.22393147]


## 抽样

从训练集、验证集、测试集中分别抽取 30、10、10 个样本，存放到 `./data/sidewalk-semantic-sample` 目录下。

In [3]:
from random import sample

# 保存路径
OUTPUT_PATH = "./data/sidewalk-semantic-sample"
image_path = os.path.join(OUTPUT_PATH, "images")
mask_path = os.path.join(OUTPUT_PATH, "masks")
os.makedirs(image_path, exist_ok=True)
os.makedirs(mask_path, exist_ok=True)

# 随机抽样
train_sample_ids = sample(train_ids, 30)
val_sample_ids = sample(val_ids, 10)
test_sample_ids = sample(test_ids, 10)
all_sample_ids = train_sample_ids + val_sample_ids + test_sample_ids

# 保存数据集列表
with open(os.path.join(OUTPUT_PATH, "train.txt"), "w") as f:
    train_sample_ids = [str(i) for i in train_sample_ids]
    f.write("\n".join(train_sample_ids))
with open(os.path.join(OUTPUT_PATH, "val.txt"), "w") as f:
    val_sample_ids = [str(i) for i in val_sample_ids]
    f.write("\n".join(val_sample_ids))
with open(os.path.join(OUTPUT_PATH, "test.txt"), "w") as f:
    test_sample_ids = [str(i) for i in test_sample_ids]
    f.write("\n".join(test_sample_ids))

# 保存标签
with open(os.path.join(OUTPUT_PATH, "labels.txt"), "w") as f:
    f.write("\n".join(labels))

# 保存图像和掩码
for id in tqdm(all_sample_ids):
    sample = dataset[id]
    image = sample["pixel_values"]
    mask = sample["label"]
    image.save(os.path.join(image_path, f"{id}.jpg"))
    mask.save(os.path.join(mask_path, f"{id}.png"))

100%|██████████| 50/50 [00:02<00:00, 20.69it/s]
